In [1]:
# One time install of dependancy
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [28]:
# Install dependancies
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import datetime as dt

from sqlalchemy import create_engine
import psycopg2
import config as creds

In [6]:
# Retreive url address from Amazon RDS and test reading of a file
url = "https://ramk-movie-bucket.s3.us-east-2.amazonaws.com/imdb_main.csv"
imdb_main2 = pd.read_csv(url)
imdb_main2.head()

,imdb_id,title,year,duration,country,language,avg_vote,votes,budget,usa_gross_income,...,g_Mystery,g_News,g_Reality_TV,g_Romance,g_Sci_Fi,g_Sport,g_Thriller,g_War,g_Western,director_list
0,tt0010680,The Sentimental Bloke,1919.0,106,Australia,English,6.3,237,NaN,NaN,...,0,0,0,1,0,0,0,0,0,['Raymond Longford']
1,tt0013579,Ombre ammonitrici,1923.0,90,Germany,German,6.8,842,NaN,NaN,...,0,0,0,0,0,0,0,0,0,['Arthur Robison']
2,tt0017938,La glace à trois faces,1983.0,45,France,"None, French",7.0,759,NaN,NaN,...,0,0,0,1,0,0,0,0,0,['Jean Epstein']
3,tt0018054,Il re dei re,1927.0,160,USA,English,7.2,1890,$ 2500000,NaN,...,0,0,0,0,0,0,0,0,0,['Cecil B. DeMille']
4,tt0018742,Il cameraman,1928.0,76,USA,English,8.1,10101,NaN,NaN,...,0,0,0,0,0,0,0,0,0,"['Edward Sedgwick', ' Buster Keaton']"


In [7]:
imdb_main2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70529 entries, 0 to 70528
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                70529 non-null  object 
 1   title                  70529 non-null  object 
 2   year                   70529 non-null  float64
 3   duration               70529 non-null  int64  
 4   country                70465 non-null  object 
 5   language               69913 non-null  object 
 6   avg_vote               70529 non-null  float64
 7   votes                  70529 non-null  int64  
 8   budget                 21107 non-null  object 
 9   usa_gross_income       14632 non-null  object 
 10  worlwide_gross_income  30375 non-null  object 
 11  metascore              12858 non-null  float64
 12  reviews_from_users     63624 non-null  float64
 13  reviews_from_critics   60360 non-null  float64
 14  release_year           70529 non-null  float64
 15  ge

In [8]:
# Drop genre column as it was broken out to each of their own columns in the clean up
imdb_main2.drop(["genre_list"], axis=1, inplace=True)

In [9]:
imdb_main2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70529 entries, 0 to 70528
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                70529 non-null  object 
 1   title                  70529 non-null  object 
 2   year                   70529 non-null  float64
 3   duration               70529 non-null  int64  
 4   country                70465 non-null  object 
 5   language               69913 non-null  object 
 6   avg_vote               70529 non-null  float64
 7   votes                  70529 non-null  int64  
 8   budget                 21107 non-null  object 
 9   usa_gross_income       14632 non-null  object 
 10  worlwide_gross_income  30375 non-null  object 
 11  metascore              12858 non-null  float64
 12  reviews_from_users     63624 non-null  float64
 13  reviews_from_critics   60360 non-null  float64
 14  release_year           70529 non-null  float64
 15  g_

In [27]:
# Use the help function to understand the parameters that may be run with 'to_sql'
help(imdb_main.to_sql)

Help on method to_sql in module pandas.core.generic:

to_sql(name: 'str', con, schema=None, if_exists: 'str' = 'fail', index: 'bool_t' = True, index_label=None, chunksize=None, dtype=None, method=None) -> 'None' method of pandas.core.frame.DataFrame instance
    Write records stored in a DataFrame to a SQL database.
    
    Databases supported by SQLAlchemy [1]_ are supported. Tables can be
    newly created, appended to, or overwritten.
    
    Parameters
    ----------
    name : str
        Name of SQL table.
    con : sqlalchemy.engine.(Engine or Connection) or sqlite3.Connection
        Using SQLAlchemy makes it possible to use any DB supported by that
        library. Legacy support is provided for sqlite3.Connection objects. The user
        is responsible for engine disposal and connection closure for the SQLAlchemy
        connectable See `here                 <https://docs.sqlalchemy.org/en/13/core/connections.html>`_.
    
    schema : str, optional
        Specify the sch

In [10]:
# run code to update capitalized letters to lowercase letters in order to write to lowercase only
# in posgres since empty database was created in advance
lowercase_columns = [col.lower() for col in imdb_main2.columns]
imdb_main2.columns = lowercase_columns
imdb_main2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70529 entries, 0 to 70528
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                70529 non-null  object 
 1   title                  70529 non-null  object 
 2   year                   70529 non-null  float64
 3   duration               70529 non-null  int64  
 4   country                70465 non-null  object 
 5   language               69913 non-null  object 
 6   avg_vote               70529 non-null  float64
 7   votes                  70529 non-null  int64  
 8   budget                 21107 non-null  object 
 9   usa_gross_income       14632 non-null  object 
 10  worlwide_gross_income  30375 non-null  object 
 11  metascore              12858 non-null  float64
 12  reviews_from_users     63624 non-null  float64
 13  reviews_from_critics   60360 non-null  float64
 14  release_year           70529 non-null  float64
 15  g_

In [29]:
# import all from config to get credentials and host and database names in postgress
from config import * 
database_string = f"postgresql://{PGUSER}:{PGPASSWORD}@{PGHOST}/{PGDATABASE}"
# create variable for the engine
engine = create_engine(database_string)


In [12]:
# write imdb_main2 to postgres file that 'already exists'
imdb_main2.to_sql("imdb_main2", engine, if_exists = 'append', index = False, chunksize = 20)

In [14]:
# get url address from AWS 'RDS' and 'S3 bucket' and read file tmdb_main
url = "https://ramk-movie-bucket.s3.us-east-2.amazonaws.com/tmdb_main.csv"
tmdb_main2 = pd.read_csv(url)
tmdb_main2.head()

,budget,imdb_id,original_language,popularity,revenue,runtime,release_year,orig_lang_cd,collection,website,...,g_Crime,g_Thriller,g_Horror,g_History,g_Mystery,g_War,g_Foreign,g_Music,g_Documentary,g_Western
0,30000000,tt0114709,en,21.946943,373554033,81.0,1995,en,yes,yes,...,0,0,0,0,0,0,0,0,0,0
1,65000000,tt0113497,en,17.015539,262797249,104.0,1995,en,no,no,...,0,0,0,0,0,0,0,0,0,0
2,0,tt0113228,en,11.712900,0,101.0,1995,en,yes,no,...,0,0,0,0,0,0,0,0,0,0
3,16000000,tt0114885,en,3.859495,81452156,127.0,1995,en,no,no,...,0,0,0,0,0,0,0,0,0,0
4,0,tt0113041,en,8.387519,76578911,106.0,1995,en,yes,no,...,0,0,0,0,0,0,0,0,0,0


In [16]:
tmdb_main2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37050 entries, 0 to 37049
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             37050 non-null  int64  
 1   imdb_id            37049 non-null  object 
 2   original_language  37044 non-null  object 
 3   popularity         37050 non-null  float64
 4   revenue            37050 non-null  int64  
 5   runtime            36836 non-null  float64
 6   release_year       37050 non-null  int64  
 7   orig_lang_cd       37050 non-null  object 
 8   collection         37050 non-null  object 
 9   website            37050 non-null  object 
 10  genre_name         34935 non-null  object 
 11  g_Animation        37050 non-null  int64  
 12  g_Comedy           37050 non-null  int64  
 13  g_Family           37050 non-null  int64  
 14  g_Adventure        37050 non-null  int64  
 15  g_Fantasy          37050 non-null  int64  
 16  g_Romance          370

In [18]:
# Drop some columns that are duplicates of what is in imdb_main2
tmdb_main2.drop(["genre_name", "g_Animation", 'g_Comedy', 'g_Family', 'g_Adventure', 'g_Fantasy', 'g_Romance', 
         'g_Drama', 'g_Action', 'g_Crime', 'g_Thriller', 'g_Horror', 'g_History', 'g_Mystery',
         'g_War', 'g_Foreign', 'g_Music', 'g_Documentary', 'g_Western'], axis=1, inplace=True)

In [20]:
tmdb_main2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37050 entries, 0 to 37049
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             37050 non-null  int64  
 1   imdb_id            37049 non-null  object 
 2   original_language  37044 non-null  object 
 3   popularity         37050 non-null  float64
 4   revenue            37050 non-null  int64  
 5   runtime            36836 non-null  float64
 6   release_year       37050 non-null  int64  
 7   orig_lang_cd       37050 non-null  object 
 8   collection         37050 non-null  object 
 9   website            37050 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 2.8+ MB


In [22]:
# write tmdb_main2 to preexisting empty database in postgres
tmdb_main2.to_sql("tmdb_main2", engine, if_exists = 'append', index = False, chunksize = 20)

In [26]:
# This method was bypassed as the join had to be done listed all the required fields as seen in
# below cells
# Write the query to join the database tables that now exist and are populated in postgres

# query = '''SELECT * FROM imdb_main2 LEFT JOIN tmdb_main2 ON imdb_main2.imdb_id = tmdb_main2.imdb_id;'''

# # Use pandas to run the query on the database, with the engine created above
# joined_tables2 = pd.read_sql_query(query, con=engine)

In [48]:
# This was dropeed due to two instances of imdb_id for columns
#joined_tables2['imdb_id']

,imdb_id,imdb_id
0,tt0010680,None
1,tt0013579,None
2,tt0017938,None
3,tt0018054,None
4,tt0018742,None
...,...,...
70524,tt9908390,None
70525,tt9911196,None
70526,tt9911774,None
70527,tt9914286,None


In [46]:
# While this was written to our databse in posgres, we will not use it as imdb_id was corupted
#joined_tables2[joined_tables2['imdb_id'].iloc[:,0].str.startswith('tt')== False]

,imdb_id,title,year,duration,country,language,avg_vote,votes,budget,usa_gross_income,...,budget,imdb_id,original_language,popularity,revenue,runtime,release_year,orig_lang_cd,collection,website


In [49]:
joined_tables2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70529 entries, 0 to 70528
Data columns (total 50 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                70529 non-null  object 
 1   title                  70529 non-null  object 
 2   year                   70529 non-null  float64
 3   duration               70529 non-null  int64  
 4   country                70465 non-null  object 
 5   language               69913 non-null  object 
 6   avg_vote               70529 non-null  float64
 7   votes                  70529 non-null  int64  
 8   budget                 21107 non-null  object 
 9   usa_gross_income       14632 non-null  object 
 10  worlwide_gross_income  30375 non-null  object 
 11  metascore              12858 non-null  float64
 12  reviews_from_users     63624 non-null  float64
 13  reviews_from_critics   60360 non-null  float64
 14  release_year           70529 non-null  float64
 15  g_

In [51]:
# wrote a small test to ensure this process will result in correctly populated imdb_id
# Write the query to join the database tables that now exist and are populated in postgres
jointest = '''SELECT imdb_main2.imdb_id, imdb_main2.title, tmdb_main2.budget FROM imdb_main2 
LEFT JOIN tmdb_main2 ON imdb_main2.imdb_id = tmdb_main2.imdb_id;'''

# Use pandas to run the query on the database, with the engine created above
join_test = pd.read_sql_query(jointest, con=engine)

In [53]:
# looks good here as imdb_db is populated
join_test.head()

,imdb_id,title,budget
0,tt0010680,The Sentimental Bloke,NaN
1,tt0013579,Ombre ammonitrici,NaN
2,tt0017938,La glace à trois faces,NaN
3,tt0018054,Il re dei re,NaN
4,tt0018742,Il cameraman,NaN


In [54]:
# write to postgres where I confirmed the imdb_db field was populate
join_test.to_sql("join_test", engine, if_exists = 'append', index = False, chunksize = 20)

In [55]:
# manually typed out all the fields that were needed for join and using sql
# Write the query to join the database tables that now exist and are populated in postgres
join = '''SELECT imdb_main2.imdb_id, imdb_main2.title, 
imdb_main2.year, imdb_main2.duration, imdb_main2.country, imdb_main2.language,
imdb_main2.avg_vote, imdb_main2.votes, imdb_main2.budget, imdb_main2.usa_gross_income,
imdb_main2.worlwide_gross_income, imdb_main2.metascore, imdb_main2.reviews_from_users, imdb_main2.reviews_from_critics,
imdb_main2.release_year, imdb_main2.g_action, imdb_main2.g_adult, imdb_main2.g_adventure,
imdb_main2.g_animation, imdb_main2.g_biography, imdb_main2.g_comedy, imdb_main2.g_crime,
imdb_main2.g_documentary, imdb_main2.g_drama, imdb_main2.g_family, imdb_main2.g_fantasy,
imdb_main2.g_history, imdb_main2.g_horror, imdb_main2.g_music, imdb_main2.g_musical,
imdb_main2.g_mystery, imdb_main2.g_news, imdb_main2.g_reality_tv, imdb_main2.g_romance,
imdb_main2.g_sci_fi, imdb_main2.g_sport, imdb_main2.g_thriller, imdb_main2.g_war,
imdb_main2.g_western, imdb_main2.director_list, 
tmdb_main2.original_language, tmdb_main2.popularity, tmdb_main2.revenue,
tmdb_main2.runtime, tmdb_main2.release_year, tmdb_main2.orig_lang_cd,
tmdb_main2.collection, tmdb_main2.website
FROM imdb_main2 
LEFT JOIN tmdb_main2 ON imdb_main2.imdb_id = tmdb_main2.imdb_id;'''

# Use pandas to run the query on the database, with the engine created above
join_final = pd.read_sql_query(join, con=engine)

In [56]:
#verify the data looks good
join_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70529 entries, 0 to 70528
Data columns (total 48 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                70529 non-null  object 
 1   title                  70529 non-null  object 
 2   year                   70529 non-null  float64
 3   duration               70529 non-null  int64  
 4   country                70465 non-null  object 
 5   language               69913 non-null  object 
 6   avg_vote               70529 non-null  float64
 7   votes                  70529 non-null  int64  
 8   budget                 21107 non-null  object 
 9   usa_gross_income       14632 non-null  object 
 10  worlwide_gross_income  30375 non-null  object 
 11  metascore              12858 non-null  float64
 12  reviews_from_users     63624 non-null  float64
 13  reviews_from_critics   60360 non-null  float64
 14  release_year           70529 non-null  float64
 15  g_

In [57]:
# verify the data looks good for final join of tables
join_final.head()

,imdb_id,title,year,duration,country,language,avg_vote,votes,budget,usa_gross_income,...,g_western,director_list,original_language,popularity,revenue,runtime,release_year,orig_lang_cd,collection,website
0,tt0010680,The Sentimental Bloke,1919.0,106,Australia,English,6.3,237,None,None,...,0,['Raymond Longford'],None,NaN,NaN,NaN,NaN,None,None,None
1,tt0013579,Ombre ammonitrici,1923.0,90,Germany,German,6.8,842,None,None,...,0,['Arthur Robison'],None,NaN,NaN,NaN,NaN,None,None,None
2,tt0017938,La glace à trois faces,1983.0,45,France,"None, French",7.0,759,None,None,...,0,['Jean Epstein'],None,NaN,NaN,NaN,NaN,None,None,None
3,tt0018054,Il re dei re,1927.0,160,USA,English,7.2,1890,$ 2500000,None,...,0,['Cecil B. DeMille'],None,NaN,NaN,NaN,NaN,None,None,None
4,tt0018742,Il cameraman,1928.0,76,USA,English,8.1,10101,None,None,...,0,"['Edward Sedgwick', ' Buster Keaton']",None,NaN,NaN,NaN,NaN,None,None,None


In [58]:
#write the joined_tables to postres to double check it is clearing imdb_id
join_final.to_sql("join_final", engine, if_exists = 'append', index = False, chunksize = 20)

In [59]:
# read data from postgres
read = 'select * from join_final'
join_table_df = pd.read_sql_query(read, con=engine)
join_table_df.head()



,imdb_id,title,year,duration,country,language,avg_vote,votes,budget,usa_gross_income,...,g_war,g_western,director_list,original_language,popularity,revenue,runtime,orig_lang_cd,collection,website
0,tt0010680,The Sentimental Bloke,1919.0,106,Australia,English,6.3,237,None,None,...,0,0,['Raymond Longford'],None,NaN,NaN,NaN,None,None,None
1,tt0013579,Ombre ammonitrici,1923.0,90,Germany,German,6.8,842,None,None,...,0,0,['Arthur Robison'],None,NaN,NaN,NaN,None,None,None
2,tt0017938,La glace à trois faces,1983.0,45,France,"None, French",7.0,759,None,None,...,0,0,['Jean Epstein'],None,NaN,NaN,NaN,None,None,None
3,tt0018054,Il re dei re,1927.0,160,USA,English,7.2,1890,$ 2500000,None,...,0,0,['Cecil B. DeMille'],None,NaN,NaN,NaN,None,None,None
4,tt0018742,Il cameraman,1928.0,76,USA,English,8.1,10101,None,None,...,0,0,"['Edward Sedgwick', ' Buster Keaton']",None,NaN,NaN,NaN,None,None,None
